In [ ]:
import os

import tensorflow as tf
import tensorflow.keras as keras
import tkinter as tk
from tkinter import filedialog

import numpy as np
import matplotlib.pyplot as plt

from PIL import Image


import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


In [ ]:
# Create a Tkinter root window (it will not be shown)
root = tk.Tk()
root.withdraw();

In [ ]:
CLASS_NAMES = ['normal', 'aom', 'ome', 'csom', 'myringosclerosis', 'earwax', 'tube']

RANDOM_STATE = 42

randomiser = np.random.RandomState(RANDOM_STATE)
np.random.seed(RANDOM_STATE)
tf.random.set_seed(RANDOM_STATE)

In [ ]:
"""
Used to resize the image based on model input size
"""
def resize_image(image, size):
    image = tf.image.resize(np.asarray(image).astype('float32'), (size, size))
    return np.asarray(image).astype('float32')

In [ ]:
"""
Creates the gradcam heatmap using last convolutional layer of the model
Code retrieved from https://keras.io/examples/vision/grad_cam/
"""

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [ ]:
import matplotlib.cm as cm
import colorsys
"""
Takes an image and calls make_gradcam_heatmap() to make a 
heatmap then superimposes it to original image
"""
def display_gradcam(img, heatmap, alpha=0.5):
    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)
    img = np.uint8(255 * img)

    
    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)
    
    si = []
    for j, y in enumerate(jet_heatmap):
        si.append([])
        for i, x in enumerate(jet_heatmap[j]):
            arr = jet_heatmap[j, i]
            hsv = colorsys.rgb_to_hsv(arr[0]/255.0, arr[1]/255.0, arr[2]/255.0)
            
            # removing non important colours
            if 150 < hsv[0]*360 < 250:
                si[j].append(arr * 0)
            else:
                si[j].append(arr * alpha)

    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = si + img

    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img, scale=True)

    # Display Grad CAM
    return superimposed_img

In [ ]:
model_path = "./models"
m = "daViT_B"

In [ ]:
model = keras.models.load_model(os.path.join(model_path, m), compile=False)
img_size = model.layers[0].input_shape[0][1]

In [ ]:
# Open a file selection dialog
selected_file = filedialog.askopenfilename(title="Select File");

# Check if a file is selected
try:
    # Now, 'selected_file' contains the path to the selected file
    image = Image.open(selected_file)

    image = np.asarray(image).astype(np.float32) / 255.0

    x_test = np.array([x for x in [image]], dtype=object)

    test_set = np.array([resize_image(x, img_size) for x in x_test])

    y_pred=model.predict(test_set, verbose=False)

    y_pred2=np.argmax(y_pred,axis=1)
    
    print(CLASS_NAMES)
    print(([float(a)*100 for a in y_pred[0]]))
    print(f"Predicted: {CLASS_NAMES[int(y_pred2)]}")

    heatmap = make_gradcam_heatmap(test_set, model, 'stack4_block2_output')

    gradcam = display_gradcam(test_set[0], heatmap, alpha=1)

    plt.figure(figsize=(10, 5))

    # Plot the original image
    plt.subplot(1, 2, 1)
    plt.imshow(test_set[0])
    plt.axis('off')
#     plt.title('Original Image')

    # Plot the GradCAM heatmap
    plt.subplot(1, 2, 2)
    plt.imshow(gradcam)
    plt.axis('off')
#     plt.title('GradCAM Heatmap')

    plt.subplots_adjust(wspace=0.01)
    # Add a caption for the combined image at the bottom
#     plt.suptitle(f"{label}", y=0.05, fontsize=16)

    plt.show()

except Exception as e:
    print(f"Error: {e}")
    

